In [2]:
import numpy as np
from src.var_processor.vpu import VPUBinaryZM
from src.tests.vpu_wrapper import VPUWrapper, signal_pre_processor

In [3]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile ----- #
# Try four VPUs with L = 4
size = 4
residual_array = np.zeros(shape=(size, 100, size))
sum_err = np.zeros(shape=(size, 100))
for i in range(0, 100):
    # Initialise two VPUs and wrappers
    data_in = np.random.randint(255, size=(size, 1))
    mean = np.asarray([128]*size).reshape(-1, 1)
    vpus = [VPUBinaryZM(size) for _ in range(0, size)]
    wrappers = [VPUWrapper(vpu) for vpu in vpus]
    for _ in range(0, 1000):
        # First VPU
        input_signal = signal_pre_processor(data_in, mean)
        for w, wrapper in enumerate(wrappers):
            _, _, residual = wrapper.iterate(input_signal)
            residual_array[:, i, w] = residual.flatten()
            input_signal = residual
    pred_sum = 0
    for wrapper in wrappers:
        pred_sum
    pred_estimate_sum = np.asarray([wrap.pred_estimate for wrap in wrappers]).sum(axis=0)
    scale_factor = np.minimum(mean, 255-mean)
    est = (mean+pred_estimate_sum*scale_factor).astype(np.uint8)
    sum_err[:, i] = (data_in - est).flatten()
    # print(f"Data: {data_in} - Estimate: {est}\n")
MSE = np.abs(sum_err.astype(np.int8)).mean(axis=1); MSE

# ----- #

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

/home/ben/projects/var_processor/src/var_processor/power_iterator.py:39: RuntimeWarning: invalid value encountered in true_divide
  self.ev = self.ev / np.linalg.norm(self.ev)
/home/ben/projects/var_processor/src/var_processor/vpu.py:212: RuntimeWarning: invalid value encountered in greater
  binary_values = np.where(np.abs(pred_inputs) > rand_val, 1, 0)



  _     ._   __/__   _ _  _  _ _/_   Recorded: 13:51:04  Samples:  94487
 /_//_/// /_\ / //_// / //_'/ //     Duration: 94.800    CPU time: 94.775
/   _/                      v3.1.3

Program: /home/ben/anaconda3/envs/var_processor/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1000/jupyter/kernel-47ca8525-9056-407b-9c53-6a974db90f3b.json

94.800 run_code  IPython/core/interactiveshell.py:3254
└─ 94.800 <module>  <ipython-input-3-796c5b4397d9>:11
   ├─ 90.140 iterate  src/tests/vpu_wrapper.py:63
   │  ├─ 46.502 add_to_array  src/tests/vpu_wrapper.py:8
   │  │  ├─ 38.779 roll  numpy/core/numeric.py:1401
   │  │  │     [10 frames hidden]  numpy
   │  │  │        32.591 [self]  
   │  │  └─ 7.723 [self]  
   │  ├─ 19.319 update_cov  src/var_processor/vpu.py:104
   │  │  ├─ 12.002 iterate  src/var_processor/power_iterator.py:25
   │  │  │  ├─ 5.054 [self]  
   │  │  │  ├─ 4.286 norm  numpy/linalg/linalg.py:2293
   │  │  │  │     [6 frames hidden]  numpy
   │  │  │  └─ 2.518

Interesting:
* Adding to the array takes up about half the time. 
* Forward takes longer than backward.
* Linalg takes 4 seconds on each power iteration.
* The non_linearity is not needed on the wrapper.
* Update cov takes the same time as the backward and forward passes - we can half iteration time by not constantly updating the cov and the eigenvector.

In [ ]:
# Generate random length 4 array
rand_vec = np.random.uniform(size=(4, 1)); rand_vec

In [ ]:
%%timeit
a = np.sqrt((rand_vec**2).sum())

In [ ]:
%%timeit
b = np.abs(rand_vec).sum()

In [ ]:
%%timeit
c = np.linalg.norm(rand_vec)

In [ ]:
a = np.sqrt((rand_vec**2).sum())
b = np.abs(rand_vec).sum()
c = np.linalg.norm(rand_vec)
print(a, b, c)

In [4]:
from src.var_processor.vpu8bit import BinaryVPU

In [5]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile ----- #
# Try four VPUs with L = 4
size = 4
residual_array = np.zeros(shape=(size, 100, size))
sum_err = np.zeros(shape=(size, 100))
for i in range(0, 100):
    # Initialise two VPUs and wrappers
    data_in = np.random.randint(254, size=(size, 1))
    mean = np.asarray([127]*size).reshape(-1, 1)
    vpus = [BinaryVPU(size) for _ in range(0, size)]
    wrappers = [VPUWrapper(vpu) for vpu in vpus]
    for _ in range(0, 1000):
        # First VPU
        input_signal = signal_pre_processor(data_in, mean)
        for w, wrapper in enumerate(wrappers):
            _, _, residual = wrapper.iterate(input_signal)
            residual_array[:, i, w] = residual.flatten()
            input_signal = residual
    pred_sum = 0
    for wrapper in wrappers:
        pred_sum
    pred_estimate_sum = np.asarray([wrap.pred_estimate for wrap in wrappers]).sum(axis=0)
    scale_factor = np.minimum(mean, 254-mean)
    est = (mean+pred_estimate_sum*scale_factor).astype(np.uint8)
    sum_err[:, i] = (data_in - est).flatten()
    # print(f"Data: {data_in} - Estimate: {est}\n")
MSE = np.abs(sum_err.astype(np.int8)).mean(axis=1); MSE

# ----- #

profiler.stop()

print(profiler.output_text(unicode=True, color=True))


  _     ._   __/__   _ _  _  _ _/_   Recorded: 13:54:01  Samples:  122883
 /_//_/// /_\ / //_// / //_'/ //     Duration: 123.346   CPU time: 123.341
/   _/                      v3.1.3

Program: /home/ben/anaconda3/envs/var_processor/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1000/jupyter/kernel-47ca8525-9056-407b-9c53-6a974db90f3b.json

123.346 run_code  IPython/core/interactiveshell.py:3254
└─ 123.346 <module>  <ipython-input-5-de7830707d60>:11
   ├─ 118.745 iterate  src/tests/vpu_wrapper.py:63
   │  ├─ 47.088 add_to_array  src/tests/vpu_wrapper.py:8
   │  │  ├─ 39.215 roll  numpy/core/numeric.py:1401
   │  │  │     [10 frames hidden]  numpy
   │  │  │        33.178 [self]  
   │  │  └─ 7.873 [self]  
   │  ├─ 31.751 update_cov  src/var_processor/vpu8bit.py:107
   │  │  ├─ 14.978 update_cov  src/var_processor/covariance8bit.py:34
   │  │  │  ├─ 12.880 [self]  
   │  │  │  └─ 2.098 recursive_update  src/var_processor/covariance8bit.py:51
   │  │  │     └─ 2.028 [se

Made it longer by 30 units! 128 vs 94. Mainly on the forward processing (22 vs 10) and update_cov (34 vs 19).

Normalise and ternary PBT first options for speed-up.

But at least now we are more accurate and better suited to convert directly to c-functions.